In [ ]:
import tarfile
import shutil
import os
import cv2
import pathlib
import numpy as np
import pandas as pd

import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab import drive
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.resnet import ResNet101

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix

# Importazione dataset

In [ ]:
%%capture
drive.mount('/content/gdrive', force_remount=True)
!tar --extract --verbose --file='/content/gdrive/MyDrive/Progetto_VIPM/dataset/train_balanced.tar'
!tar --extract --verbose --file='/content/gdrive/MyDrive/Progetto_VIPM/dataset/val.tar'
!unzip '/content/gdrive/MyDrive/Progetto_VIPM/dataset/val_degraded.zip' -d '/content/'

## Train

In [ ]:
%%capture
csv_train_file = pd.read_csv("/content/gdrive/MyDrive/Progetto_VIPM/annot/train_balanced_info.csv", dtype=str)
csv_train_file.columns = ['filename', 'label']
parent_dir = "train_set/"
labels = csv_train_file['label']
for label in labels:
    path = os.path.join(parent_dir, label)
    os.makedirs(path, exist_ok=True)    
for _, row in csv_train_file.iterrows():
    label = row['label']
    path = os.path.join(parent_dir, row['filename'])
    img_name = os.path.split(path)[-1]
    new_path = os.path.join(parent_dir, label, img_name)
    shutil.copy(path, new_path)

In [ ]:
train_data_dir = pathlib.Path('train_set/')
seed_train_validation = 42 # must be same for train_ds and val_ds
shuffle_value = True
validation_split = 0.2
batch_size = 512

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    labels="inferred",
    label_mode="categorical",
    validation_split=validation_split,
    subset="training",
    seed=seed_train_validation,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    labels="inferred",
    label_mode="categorical",
    validation_split=validation_split,
    subset="validation",
    seed=seed_train_validation,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

Found 125499 files belonging to 251 classes.
Using 100400 files for training.
Found 125499 files belonging to 251 classes.
Using 25099 files for validation.


## Validation

In [ ]:
%%capture
csv_test_file = pd.read_csv("/content/gdrive/MyDrive/Progetto_VIPM/annot/val_info.csv", dtype=str)
csv_test_file.columns = ['filename', 'label']
parent_dir = "val_set/"
labels = csv_test_file['label']
for label in labels:
    path = os.path.join(parent_dir, label)
    os.makedirs(path, exist_ok=True)    
for _, row in csv_test_file.iterrows():
    label = row['label']
    path = os.path.join(parent_dir, row['filename'])
    img_name = os.path.split(path)[-1]
    new_path = os.path.join(parent_dir, label, img_name)
    print(new_path)
    shutil.copy(path, new_path)

In [ ]:
%%capture
csv_test_degraded_file = pd.read_csv("/content/gdrive/MyDrive/Progetto_VIPM/annot/val_info.csv", dtype=str)
csv_test_degraded_file.columns = ['filename', 'label']
parent_dir = "val_set_degraded/"
labels = csv_test_degraded_file['label']
for label in labels:
    path = os.path.join(parent_dir, label)
    os.makedirs(path, exist_ok=True)    
for _, row in csv_test_file.iterrows():
    label = row['label']
    path = os.path.join(parent_dir, row['filename'])
    img_name = os.path.split(path)[-1]
    new_path = os.path.join(parent_dir, label, img_name)
    print(new_path)
    shutil.copy(path, new_path)

In [ ]:
test_data_dir = pathlib.Path('val_set/')
test_degraded_data_dir = pathlib.Path('val_set_degraded/')

shuffle_value = True

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_data_dir,
    labels="inferred",
    label_mode="categorical",
    subset=None,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

test_degraded_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_degraded_data_dir,
    labels="inferred",
    label_mode="categorical",
    subset=None,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

Found 11993 files belonging to 251 classes.
Found 11993 files belonging to 251 classes.


# Creazione modello

In [ ]:
base_eff_model = EfficientNetB0(weights="imagenet", include_top=True, input_shape=(224,224,3))
base_eff_model.summary()

21834768/21834768 [==============================] - 3s 0us/step
Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 tf.math.truediv (TF

In [ ]:
trainable_layer = False
for layer in base_eff_model.layers:
    if layer.name == "block7a_expand_conv":
        trainable_layer=True
    layer.trainable=trainable_layer

eff_model = Model(inputs=base_eff_model.input, outputs=base_eff_model.get_layer('top_dropout').output)
last_layer = base_eff_model.get_layer('top_dropout').output
flatten_layer = Flatten()(last_layer)
classification_layer = Dense(251, activation='softmax')(flatten_layer)
eff_model = Model(inputs=eff_model.input, outputs=classification_layer)
# eff_model.summary()

In [ ]:
eff_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3)])
history = eff_model.fit(train_ds, epochs=10, verbose=1, validation_data=val_ds)
eff_model.save('/content/gdrive/MyDrive/Progetto_VIPM/models/effNetB0_balanced.h5')

Epoch 1/10
197/197 [==============================] - 301s 1s/step - loss: 3.0529 - accuracy: 0.3499 - top_k_categorical_accuracy: 0.5219 - val_loss: 2.2816 - val_accuracy: 0.4718 - val_top_k_categorical_accuracy: 0.6623
Epoch 2/10
197/197 [==============================] - 272s 1s/step - loss: 2.0580 - accuracy: 0.5155 - top_k_categorical_accuracy: 0.7020 - val_loss: 2.0611 - val_accuracy: 0.5161 - val_top_k_categorical_accuracy: 0.7034
Epoch 3/10
197/197 [==============================] - 271s 1s/step - loss: 1.7402 - accuracy: 0.5775 - top_k_categorical_accuracy: 0.7575 - val_loss: 1.9711 - val_accuracy: 0.5380 - val_top_k_categorical_accuracy: 0.7184
Epoch 4/10
197/197 [==============================] - 268s 1s/step - loss: 1.4984 - accuracy: 0.6290 - top_k_categorical_accuracy: 0.8004 - val_loss: 1.9591 - val_accuracy: 0.5442 - val_top_k_categorical_accuracy: 0.7255
Epoch 5/10
197/197 [==============================] - 272s 1s/step - loss: 1.3041 - accuracy: 0.6701 - top_k_categor

In [ ]:
history = eff_model.fit(train_ds, epochs=5, verbose=1, validation_data=val_ds)
eff_model.save('/content/gdrive/MyDrive/Progetto_VIPM/models/effNetB0_balanced.h5')

Epoch 1/5
197/197 [==============================] - 270s 1s/step - loss: 0.5889 - accuracy: 0.8394 - top_k_categorical_accuracy: 0.9485 - val_loss: 2.1569 - val_accuracy: 0.5538 - val_top_k_categorical_accuracy: 0.7314
Epoch 2/5
197/197 [==============================] - 270s 1s/step - loss: 0.5235 - accuracy: 0.8559 - top_k_categorical_accuracy: 0.9590 - val_loss: 2.2047 - val_accuracy: 0.5554 - val_top_k_categorical_accuracy: 0.7305
Epoch 3/5
197/197 [==============================] - 268s 1s/step - loss: 0.4657 - accuracy: 0.8718 - top_k_categorical_accuracy: 0.9662 - val_loss: 2.2852 - val_accuracy: 0.5471 - val_top_k_categorical_accuracy: 0.7287
Epoch 4/5
197/197 [==============================] - 268s 1s/step - loss: 0.4193 - accuracy: 0.8841 - top_k_categorical_accuracy: 0.9717 - val_loss: 2.2918 - val_accuracy: 0.5519 - val_top_k_categorical_accuracy: 0.7294
Epoch 5/5
197/197 [==============================] - 268s 1s/step - loss: 0.3760 - accuracy: 0.8952 - top_k_categorical_

In [ ]:
eff_model = keras.models.load_model('/content/gdrive/MyDrive/Progetto_VIPM/models/effNetB0_balanced.h5')
eff_model.evaluate(test_ds, batch_size=512)
eff_model.evaluate(test_degraded_ds, batch_size=512)

24/24 [==============================] - 30s 1s/step - loss: 5.7149 - accuracy: 0.2020 - top_k_categorical_accuracy: 0.3299


[5.714931964874268, 0.20203451812267303, 0.32985907793045044]